# Camera Calibration & AprilTag Detection Demo

This Jupyter Notebook provides a complete guide to AprilTag recognition using computer vision techniques. It covers how to print an AprilTag, perform camera calibration with `OpenCV`, and detect tags in captured images. 

To explore the demo with default files and settings, enable test mode below. If test mode is enabled, you can skip all parameter modifications in this demo.

In [ ]:
# @title Configuration
# @markdown Set `test = True` to use default files and parameters.
test = True  # @param {type:"boolean"}

## Requirements

This notebook requires the following files and libraries.

In [ ]:
# @title Install Requirements

# only install if we are running in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install pupil-apriltags opencv-python matplotlib numpy

---

## Hardware Preparation

### 1. Print a Checkerboard Pattern

To calibrate your camera, you will need a printed checkerboard pattern. You can download a standard A4 checkerboard from [here](https://raw.githubusercontent.com/opencv/opencv/master/doc/pattern.png) or [this PDF version](https://calib.io/pages/camera-calibration-pattern-generator). Print it out and attach it to a flat surface.


### 2. Get AprilTag Images
You can download an example AprilTag [tag41_12_00110](https://drive.google.com/file/d/15vwvnacpYkXJLtxag8rbLcZLsbhWkwwo/view?usp=sharing), or follow the steps below to generate a custom tag with your desired size and family. If you choose to use the example tag, you can jump to Camera Calibration


#### Select AprilTag images

You can find the images for the tags in the [apriltag-imgs repo](https://github.com/AprilRobotics/apriltag-imgs). 

> Note: "For the vast majority of applications, the tagStandard41h12 family will be the correct choice. Scale up the images in your favorite editor and print them out or use the helper function below to automatically generate a printable SVG version of the tag at your desired size.
>
> Some heuristics for when to choose other tag families:
>
> 1. If you need more tags, use tagStandard52h13
> 2. If you need to maximize the use of space on a small circular object, use tagCircle49h12 (or tagCircle21h7).
> 3. If you want to make a recursive tag use tagCustom48h12.
> 4. If you want compatibility with the ArUcO detector use tag36h11
>
> If none of these fit your needs, generate your own custom tag family [here](https://github.com/AprilRobotics/apriltag-generation)."

— *Source: [AprilTag GitHub README – Choosing a Tag Family](https://github.com/AprilRobotics/apriltag?tab=readme-ov-file#choosing-a-tag-family)*

#### Get printable AprilTags

This is a helper function for generating a scalable SVG version of an AprilTag from a PNG image. 

- `tag`: the full name of the tag, e.g., `"tag41_12_00110"`.

- `tag_family`: the tag family the tag belongs to, e.g., `"tagStandard41h12"`.

In [ ]:
# @title Generate SVG AprilTag
tag = "tag41_12_00110"#@param {type:"string"}
tag_family = "tagStandard41h12"  # @param ["tag16h5", "tag25h9", "tag36h11", "tagCircle21h7", "tagCircle49h12", "tagCustom48h12", "tagStandard41h12", "tagStandard52h13"]

import os
import requests
from PIL import Image
from IPython.display import SVG, display

def generate_apriltag_svg(tag_name: str, tag_family: str = "tagStandard41h12", size_mm: int = 60):
    """
    Download an AprilTag PNG and convert it to SVG with a given physical size.
    """
    filename = f"{tag_name}.png"
    svg_filename = f"{tag_name}_{size_mm}mm.svg"
    png_url = f"https://raw.githubusercontent.com/AprilRobotics/apriltag-imgs/master/{tag_family}/{filename}"

    # Download tag PNG if not already downloaded
    if not os.path.exists(filename):
        r = requests.get(png_url)
        if r.status_code == 200:
            with open(filename, "wb") as f:
                f.write(r.content)
        else:
            raise ValueError(f"Failed to download image. Status code: {r.status_code}")

    # Convert PNG to SVG
    def gen_rgba(rbga):
        (_r, _g, _b, _raw_a) = rbga
        _a = _raw_a / 255
        return f'rgba({_r}, {_g}, {_b}, {_a})'

    def gen_gridsquare(row_num, col_num, pixel):
        _rgba = gen_rgba(pixel)
        _id = f'box{row_num}-{col_num}'
        return f'\t<rect width="1" height="1" x="{row_num}" y="{col_num}" fill="{_rgba}" id="{_id}"/>\n'

    def gen_apriltag_svg(width, height, pixel_array, size):
        svg_text = '<?xml version="1.0" standalone="yes"?>\n'
        svg_text += f'<svg width="{size}" height="{size}" viewBox="0,0,{width},{height}" xmlns="http://www.w3.org/2000/svg">\n'
        for _y in range(height):
            for _x in range(width):
                svg_text += gen_gridsquare(_x, _y, pixel_array[_x, _y])
        svg_text += '</svg>\n'
        return svg_text

    with Image.open(filename, 'r') as im:
        width, height = im.size
        pix_vals = im.load()
        svg_text = gen_apriltag_svg(width, height, pix_vals, f"{size_mm}mm")

    with open(svg_filename, 'w') as fp:
        fp.write(svg_text)

    if os.path.exists(svg_filename):
        display(SVG(filename=svg_filename))
    else:
        raise FileNotFoundError(f"{svg_filename} not generated.")

    
generate_apriltag_svg(tag, tag_family)

You can convert the generated PDF file into a .png image and insert it into a document editor (e.g., Microsoft Word, Pages, or Google Docs) to adjust its printed size as needed.

---

## Camera Calibration

### 1. Camera Calibration Photo Guide

Camera calibration requires using the printed checkerboard and the camera you want to calibrate. Please follow the guidelines below when capturing images of the checkerboard:

- Use the same printed checkerboard for all photos.
- Place the checkerboard on a flat and rigid surface to avoid distortion.
- Use even lighting and make sure the checkerboard is sharp, unobstructed, and fully within the frame。
- Capture checkerboard images from a variety of positions, distances, and angles to improve calibration accuracy.
- Capture at least 10 images, ideally 15–20, from different angles and distances.
- Save all images in a consistent format such as .jpg or .png, and ensure that all images have the same resolution.

After calibration, store all images in one folder.

We provide a set of hard-coded checkerboard images in a [ZIP file](checkerboard_images.zip), manually captured for camera calibration purposes. Below are two sample images from the set:
<p align="center">
  <img src="https://images.weserv.nl/?url=drive.google.com/uc?id=1zcOI8qm66jzLp7icGdK0QkAoDe6242LN" width="500"/>
  <img src="https://images.weserv.nl/?url=drive.google.com/uc?id=1VQm7praOF-hAJWy9hWhg3dnKDokBXhug" width="500"/>
</p>





### 2. Run Camera Calibration with Checkerboard Images

Run the cell below to either upload your own checkerboard images or use the pre-supplied ones.

In [ ]:
# @title Get Checkerboard Images
# @markdown Upload your own checkerboard images or use the pre-supplied one if `test` mode is enabled.

import os
from google.colab import files

image_dir = "/content/checkerboard_images"
os.makedirs(image_dir, exist_ok=True)

if test:
    !wget -O checkerboard_images.zip https://raw.githubusercontent.com/AccelerationConsortium/ac-training-lab/apriltag_demo/src/ac_training_lab/apriltag_demo/checkerboard_images.zip
    !unzip -o -j checkerboard_images.zip -d {image_dir}
else:
    uploaded = files.upload()
    for name in uploaded.keys():
        os.rename(name, os.path.join(image_dir, name))

image_paths = [
    os.path.join(image_dir, f)
    for f in sorted(os.listdir(image_dir))
    if f.lower().endswith(('.jpg', '.png'))
]


You should modify the following four parameters to calibrate your camera.

- rows: the number of inner corners in the vertical direction.

- cols: the number of inner corners in the horizontal direction.

> Note: These refer to the number of corner intersections, not the number of squares.

- size: the length of one square on the checkerboard, in millimeters.

In [ ]:
# @title Calibrate Camera
rows = 6  # @param {type:"integer"}
cols = 9  # @param {type:"integer"}
square_size = 23  # @param {type:"number"}

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from typing import List, Tuple
import matplotlib.pyplot as plt

def calibrate_camera_from_images(
    image_paths: List[str],
    rows: int,
    cols: int,
    square_size: float,
    show_detections: bool = False,
    save_path: str = None
) -> Tuple[np.ndarray, Tuple[float, float, float, float]]:
    pattern_size = (cols, rows) if rows < cols else (rows, cols)
    
    x = np.arange(pattern_size[0]) * square_size
    y = np.arange(pattern_size[1]) * square_size
    xgrid, ygrid = np.meshgrid(x, y)
    zgrid = np.zeros_like(xgrid)
    opoints = np.dstack((xgrid, ygrid, zgrid)).reshape((-1, 1, 3)).astype(np.float32)

    imagesize = None
    ipoints = []

    for filename in image_paths:
        rgb = cv2.imread(filename)
        if rgb is None:
            print(f'Error loading {filename}, skipping.')
            continue

        if imagesize is None:
            imagesize = (rgb.shape[1], rgb.shape[0])
        else:
            assert imagesize == (rgb.shape[1], rgb.shape[0]), "Inconsistent image sizes."

        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY) if len(rgb.shape) == 3 else rgb
        found, corners = cv2.findChessboardCorners(gray, pattern_size)

        if show_detections:
            display_img = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            cv2.drawChessboardCorners(display_img, pattern_size, corners, found)
            cv2_imshow(display_img)
            plt.close('all') 

        if found:
            ipoints.append(corners)
        else:
            print(f'No corners found in {filename}, skipping.')

    if len(ipoints) < 1:
        raise ValueError("No valid images with detected corners found.")

    flags = (cv2.CALIB_ZERO_TANGENT_DIST |
             cv2.CALIB_FIX_K1 |
             cv2.CALIB_FIX_K2 |
             cv2.CALIB_FIX_K3 |
             cv2.CALIB_FIX_K4 |
             cv2.CALIB_FIX_K5 |
             cv2.CALIB_FIX_K6)

    retval, K, dcoeffs, rvecs, tvecs = cv2.calibrateCamera(
        [opoints] * len(ipoints),
        ipoints,
        imagesize,
        cameraMatrix=None,
        distCoeffs=np.zeros(5),
        flags=flags
    )

    fx, fy, cx, cy = K[0,0], K[1,1], K[0,2], K[1,2]
    if save_path:
        np.save(save_path, np.array([fx, fy, cx, cy]))
        print(f'Saved camera parameters to {save_path}')

    print("\nCamera intrinsics (units: pixels):")
    print(f"  fx = {fx:.3f}")
    print(f"  fy = {fy:.3f}")
    print(f"  cx = {cx:.3f}")
    print(f"  cy = {cy:.3f}")
    print(f"\nPython copy-paste: fx, fy, cx, cy = {(fx, fy, cx, cy)}")

    return K, (fx, fy, cx, cy)

K, (fx, fy, cx, cy) = calibrate_camera_from_images(
    image_paths=image_paths,
    rows=rows,
    cols=cols,
    square_size=square_size,
    show_detections=True,
    save_path="/content/camera_params.npy"
)

---

## AprilTag Detection

Run the cell below to either upload your own AprilTag image or use the pre-supplied ones.

In [ ]:
# @title Get AprilTag Image
# @markdown Upload your own AprilTag image or use the pre-supplied one if `test` mode is enabled.

if test:
    !wget -O apriltag_example.jpg https://raw.githubusercontent.com/AccelerationConsortium/ac-training-lab/apriltag_demo/src/ac_training_lab/apriltag_demo/apriltag_test/apriltag_test.jpg
    apriltag_path = "/content/apriltag_test.jpg"
else:
    uploaded = files.upload()
    if len(uploaded) == 0:
        raise ValueError("No image uploaded.")
    apriltag_filename = next(iter(uploaded))
    apriltag_path = os.path.join("/content", apriltag_filename)


You should modify the following three parameters to recongnize the apriltag.  

- `families`: the tag family the AprilTag belongs to (e.g., `tag16h5`, `tagStandard41h12`, `tag36h11`, etc.).

- `tagsize`: The size of the tag in centimeters. 

> Note: "Each tag design has a black border and a white border, but some designs have the white border on the inside and some have the black border on the inside. The tagsize is thus measured from where the two borders meet, see the figure below for an example. The tag size should not be measured from the outside of the tag. The tag size is defined as the distance between the detection corners, or alternately, the length of the edge between the white border and the black border. The following illustration marks the detection corners with red Xs and the tag size with a red arrow for a tag from the 48h12Custom tag family."

— *Source: [AprilTag GitHub README – Pose Estimation](https://github.com/AprilRobotics/apriltag?tab=readme-ov-file#pose-estimation)*
<p align="center">
  <img src="https://raw.githubusercontent.com/AprilRobotics/apriltag/master/tag_size_48h12.png" alt="AprilTag Example" width="400">
</p>

We provide an example AprilTag image [here](apriltag_test\apriltag_test.jpg)

In [ ]:
# @title AprilTag Detection
# @markdown Configure detection parameters and run detection on the image.

families = "tagStandard41h12"  # @param ["tag16h5", "tag25h9", "tag36h11", "tagCircle21h7", "tagCircle49h12", "tagCustom48h12", "tagStandard41h12", "tagStandard52h13"]
tagsize = 4.5 # @param {type:"number"}

import cv2
import numpy as np
from pupil_apriltags import Detector
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

image = cv2.imread(apriltag_path, cv2.IMREAD_GRAYSCALE)
if image is None:
    raise ValueError(f"Failed to load image at {apriltag_path}")

camera_params = np.load("/content/camera_params.npy")

at_detector = Detector(
    families=families,
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

detections = at_detector.detect(
    image,
    estimate_tag_pose=True,
    camera_params=camera_params,
    tag_size=tagsize
)

print(f"Found {len(detections)} tags")

for det in detections:
    tag_id = det.tag_id
    center = det.center
    corners = det.corners.astype(int)

    cv2.polylines(image, [corners], True, 255, 2)
    cx, cy = map(int, center)
    cv2.circle(image, (cx, cy), 4, 255, -1)
    cv2.putText(image, str(tag_id), (cx, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

    print(f"\nTag ID: {tag_id}")
    print(f"  Center: {center}")
    print(f"  Corners: {det.corners}")

    if det.pose_t is not None:
        t = det.pose_t.flatten()
        print(f"  Translation (x, y, z): {t}")
        print(f"  Distance from camera: {np.linalg.norm(t):.2f} meters")

        euler = R.from_matrix(det.pose_R).as_euler('xyz', degrees=True)
        roll, pitch, yaw = euler
        print(f"  Roll: {roll:.2f}°, Pitch: {pitch:.2f}°, Yaw: {yaw:.2f}°")
    else:
        print("No pose estimated (pose_t is None)")

plt.imshow(image, cmap='gray')
plt.title("AprilTag Detection")
plt.show()

### Output Field Explanation

- **Tag ID**: The unique identifier of the detected AprilTag.

- **Center**: The (x, y) pixel coordinates of the tag's center in the image.

- **Corners**: The pixel coordinates of the tag’s four corners, typically ordered counterclockwise.

The camera coordinate system follows OpenCV's default convention:

- x-axis: right  
- y-axis: down  
- z-axis: forward (into the scene)

- **Translation (x, y, z)**: The tag’s 3D position relative to the camera, in centimeters:
  - x: horizontal offset (left/right)
  - y: vertical offset (up/down)
  - z: distance from the camera (depth)

- **Distance from camera**: The Euclidean distance between the camera and the tag’s center in 3D space.

- **Roll, Pitch, Yaw**: The tag’s rotation angles relative to the camera, in degrees:
  - Roll: rotation around the x-axis
  - Pitch: rotation around the y-axis
  - Yaw: rotation around the z-axis
